In [ ]:
import ipywidgets.widgets as widgets
from time import sleep
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.camera import get_images_and_cameras
from ballsbot.controller import link_controller
from ballsbot.utils import keep_rps, run_as_thread
from ballsbot.augmented_lidar import get_augmented_lidar
from ballsbot.ros_messages import get_ros_messages
from ballsbot.manipulator import Manipulator
from ballsbot.joystick import JoystickWrapper
from ballsbot.drawing import ManipulatorPoseDrawing, BotPoseSelfCoordsDrawing, StatusMessagesDrawing
from ballsbot.dashboard import Dashboard
from ballsbot.lidar_drawing import LidarDrawing
from ballsbot.ups import UPS

In [ ]:
get_ros_messages().start(sync=False)

In [ ]:
lidar = get_augmented_lidar()
lidar.start()

# lidar = Lidar()
# lidar.calibrate()
# print(lidar.calibration)

ups = UPS()

In [ ]:
controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

images = []
for image, camera in get_images_and_cameras():
    images.append(image)
hbox = widgets.HBox(images)
display(hbox)

dashboard = Dashboard(widgets)

lidar_drawing_raw = BotPoseSelfCoordsDrawing(dashboard, 'lidar')
lidar_drawing = LidarDrawing(lidar, lidar_drawing_raw)
dashboard.add_subplot('lidar', lidar_drawing_raw.get_drawing_func_name())

status_drawing_raw = StatusMessagesDrawing(dashboard, 'status')
dashboard.add_subplot('status', status_drawing_raw.get_drawing_func_name())

manipulator_drawing_raw = ManipulatorPoseDrawing(dashboard, 'manipulator_xy', 'manipulator_xz')
dashboard.add_subplot('manipulator_xy', manipulator_drawing_raw.get_drawing_func_name())
dashboard.add_subplot('manipulator_xz', manipulator_drawing_raw.get_drawing_func_name())

display(dashboard.get_image())

In [ ]:
joystick_wrapper = JoystickWrapper(controller)
manipulator = Manipulator(joystick_wrapper, without_encoders=True)
link_controller(joystick_wrapper)

In [ ]:
def update_images():
    ts = None
    while True:
        ts = keep_rps(ts, fps=1)

        lidar_drawing.update_image_once(cached=False)

        capacity = int(round(ups.get_capacity()))
        distances = lidar.cached_distances
        if distances and 'manipulator' in distances and distances['manipulator']:
            distance = distances['manipulator'].get('distance', -1)
        else:
            distance = -1
        status_drawing_raw.update_image(f'battery charge: {capacity}%\ntarget: {distance} m')

        pose = manipulator.get_manipulator_pose()
        manipulator_drawing_raw.update_image(pose, override_crop_half_size=750.)

        dashboard.redraw()

run_as_thread(update_images)

In [ ]:
# from ballsbot.utils import join_all_theads
# join_all_theads()